# Math Question Answer Verification Competition

## Starter Code

Borrowed from [official Unsloth implementation](https://colab.research.google.com/drive/1Ys44kVvmeZtnICzWz0xgpRnrIOjZAuxp?usp=sharing#scrollTo=MKX_XKs_BNZR)

In [1]:
# %%capture
# This cell will take time
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.8/161.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 119.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 95.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 8bit quantization to reduce memory usage. Can be False.


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [3]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2024.11.5: Fast Llama patching. Transformers = 4.46.2.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu124. CUDA = 8.0. CUDA Toolkit = 12.4.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

## Load model and wrap with LoRA adapters

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = True,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.11.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


## Competition dataset

In [5]:
# download and load competition dataset

from datasets import load_dataset
dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp")
# print and see dataset
dataset

README.md:   0%|          | 0.00/2.09k [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/195M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/195M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/3.65M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'is_correct', 'answer', 'solution'],
        num_rows: 1000000
    })
    test: Dataset({
        features: ['question', 'is_correct', 'answer', 'solution'],
        num_rows: 10000
    })
})

In [6]:
prompt = """You are a great mathematician and you are tasked with finding if an answer to a given maths question is correct or not. Yout response should be 'True' if correct, otherwise 'False'. Below is Question and Answer.



### Question:
{}

### Answer:
{}

### Explainaition

### Output:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    question = examples["question"]
    ans       = examples["answer"]
    output      = examples["is_correct"]
    texts = []
    for instruction, input, output in zip(question, ans, output):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }




In [7]:
# Process the training dataset and generate prompt for each datapoint

train_dataset = dataset['train'].map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/1000000 [00:00<?, ? examples/s]

In [8]:
num_examples = len(train_dataset)
print(f"Number of examples in the dataset: {num_examples}")

Number of examples in the dataset: 1000000


In [9]:
#print a sample training example
train_dataset['text'][1]

"You are a great mathematician and you are tasked with finding if an answer to a given maths question is correct or not. Yout response should be 'True' if correct, otherwise 'False'. Below is Question and Answer.\n\n\n\n### Question:\nIf $x + y = 16$ and $x-y = 2$, what is the value of $x^2 - y^2$?\n\n### Answer:\n32\n\n### Explainaition\n\n### Output:\nTrue<|end_of_text|>"

In [10]:
import wandb

In [11]:
wandb.login()

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [12]:
sweep_config = {
    'method': 'random'
    }

In [13]:
metric = {
    'name': 'loss',
    'goal': 'minimize'
    }

sweep_config['metric'] = metric

In [14]:
parameters_dict = {
        "per_device_train_batch_size": {
            "values": [2, 4, 8]
        },
        "gradient_accumulation_steps": {
            "values": [2, 4, 8]
        },
        "max_steps": {
            "values": [10, 20, 50]
        },
        "warmup_steps": {
            "values": [5, 10, 20]
        },
        "num_train_epochs": {
            "value": 1  # Fixed parameter
        },
        "learning_rate": {
            "distribution": "log_uniform_values",
            "min": 1e-5,
            "max": 1e-3
        },
        "weight_decay": {
            "distribution": "uniform",
            "min": 0.0,
            "max": 0.1
        },
        "lr_scheduler_type": {
            "values": ["linear", "cosine", "cosine_with_restarts"]
        },
        "optim": {
            "values": [
                "adamw_8bit",
                "adamw_bnb_8bit",
                "adamw_torch",
                "adafactor",
                "adamw_hf"
            ]
        }
    }



sweep_config['parameters'] = parameters_dict

In [15]:
import pprint
pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'minimize', 'name': 'loss'},
 'parameters': {'gradient_accumulation_steps': {'values': [2, 4, 8]},
                'learning_rate': {'distribution': 'log_uniform_values',
                                  'max': 0.001,
                                  'min': 1e-05},
                'lr_scheduler_type': {'values': ['linear',
                                                 'cosine',
                                                 'cosine_with_restarts']},
                'max_steps': {'values': [10, 20, 50]},
                'num_train_epochs': {'value': 1},
                'optim': {'values': ['adamw_8bit',
                                     'adamw_bnb_8bit',
                                     'adamw_torch',
                                     'adafactor',
                                     'adamw_hf']},
                'per_device_train_batch_size': {'values': [2, 4, 8]},
                'warmup_steps': {'values': [5, 10, 20]},
       

In [16]:
sweep_id = wandb.sweep(sweep_config, project="kaggle sweeps")

Create sweep with ID: gxnz7ob2
Sweep URL: https://wandb.ai/garima440-new-york-university/kaggle%20sweeps/sweeps/gxnz7ob2


## SFT

In [17]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
import os

def train(config=None):
    global model
    # Initialize a new wandb run
    with wandb.init(config=config) as run:
        # If called by wandb.agent, this config will be set by Sweep Controller
        config = wandb.config


        # Create a unique output directory for each run
        output_dir = os.path.join("outputs", run.name)
        os.makedirs(output_dir, exist_ok=True)

        # 1. Enable gradient checkpointing
        model.gradient_checkpointing_enable()

        # 5. Enable distributed training if multiple GPUs are available
        if torch.cuda.device_count() > 1:
            model = torch.nn.DataParallel(model)

        # 2. Configure mixed precision training
        dtype = torch.bfloat16 if is_bfloat16_supported() else torch.float16

        # 3. Optimize memory usage
        torch.cuda.empty_cache()

        training_args = TrainingArguments(
            per_device_train_batch_size=config.per_device_train_batch_size,
            gradient_accumulation_steps=config.gradient_accumulation_steps,
            warmup_steps=config.warmup_steps,
            num_train_epochs=1,
            max_steps=config.max_steps,
            learning_rate=config.learning_rate,
            fp16=not is_bfloat16_supported(),
            bf16=is_bfloat16_supported(),
            logging_steps=1,
            optim=config.optim,
            weight_decay=config.weight_decay,
            lr_scheduler_type=config.lr_scheduler_type,
            seed=3407,
            output_dir=output_dir,
            report_to="wandb",  # Enable WandB reporting
            run_name=run.name,  # Use WandB run name

            # Added optimization parameters
            gradient_checkpointing=True,
            #tf32=True,  # Enable TensorFloat-32 on Ampere GPUs
            dataloader_num_workers=4,  # Parallel data loading
            dataloader_pin_memory=True,  # Pin memory for faster data transfer
            torch_compile=True,  # Enable PyTorch 2.0 compilation

        )




        trainer = SFTTrainer(
            model=model,  # Make sure model is defined or passed as parameter
            tokenizer=tokenizer,  # Make sure tokenizer is defined or passed as parameter
            train_dataset=train_dataset,  # Make sure dataset is defined or passed as parameter
            dataset_text_field="text",
            max_seq_length=max_seq_length,  # Make sure this is defined or passed as parameter
            dataset_num_proc=8,
            packing=True,
            args=training_args,
            data_collator=None,  # Let DataLoader handle collation
        )



        # # Train and get stats
        # trainer_stats = trainer.train()

        # # Log metrics to WandB
        # metrics = {
        #     "loss": trainer_stats.metrics['train_loss']}
        # wandb.log(metrics)

        # return trainer_stats.metrics['train_loss']
        # Train and get stats
        trainer_stats = trainer.train()

        metrics = {
            "loss": trainer_stats.metrics['train_loss'],
        }
        wandb.log(metrics)
        print(f"final_loss: {trainer_stats.metrics['train_loss']}")





In [18]:
wandb.agent(sweep_id, train, count=30)

wandb: Agent Starting Run: dj41h4cj with config:
wandb: 	gradient_accumulation_steps: 4
wandb: 	learning_rate: 1.330534730680917e-05
wandb: 	lr_scheduler_type: linear
wandb: 	max_steps: 20
wandb: 	num_train_epochs: 1
wandb: 	optim: adamw_hf
wandb: 	per_device_train_batch_size: 8
wandb: 	warmup_steps: 5
wandb: 	weight_decay: 0.02286321352217939
wandb: Currently logged in as: garima440 (garima440-new-york-university). Use `wandb login --relogin` to force relogin


Generating train split: 0 examples [00:00, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 64,445 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 8 | Gradient Accumulation steps = 4
\        /    Total batch size = 32 | Total steps = 20
 "-____-"     Number of trainable parameters = 83,886,080


Unsloth: Enabled auto compiling


wandb: WARNING Config item 'per_device_train_batch_size' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'gradient_accumulation_steps' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'weight_decay' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'num_train_epochs' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'max_steps' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'lr_scheduler_type' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'warmup_steps' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'optim' was locked by 'sweep' (ignored update).


...

Unsloth: Enabled auto compiling


wandb: WARNING Config item 'per_device_train_batch_size' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'gradient_accumulation_steps' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'weight_decay' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'num_train_epochs' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'max_steps' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'lr_scheduler_type' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'warmup_steps' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'optim' was locked by 'sweep' (ignored update).


Step,Training Loss
1,0.126700
2,0.103400
3,0.070400
4,0.070700
5,0.098900
6,0.119100
7,0.125800
8,0.116500
9,0.129600
10,0.112400


final_loss: 0.22409524410963058


loss,▁
train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇████
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇█████
train/grad_norm,▆▄▂▁▃▅▄▄▄▄▆▆▇▆▅▇▇▇▆█▇█▆▇██▆▇▆▆▆▅▅▆▆▆▆▆▆▇
train/learning_rate,▂▂▃▄▅▇▇██████▇▇▇▇▇▆▆▆▅▅▅▅▄▄▃▃▃▂▂▂▂▂▁▁▁▁▁
train/loss,▂▂▁▁▂▂▂▂▂▂▂▃▃▃▃▃▄▃▄▄▅▄▄▄▄▄▄▄▄▄▄▄▅▆▅▆▇▇▆█
loss,0.2241
total_flos,7.46010452164608e+16
train/epoch,0.01241
train/global_step,50
train/grad_norm,0.58643


In [ ]:
# trainer_stats = trainer.train()

## inference

In [ ]:
# Sample inference data point
test_dataset = dataset['test']

sample_ques = test_dataset['question'][0]
sample_ans = test_dataset['answer'][0]


In [ ]:
# Running inference on single test
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
input_prompt = prompt.format(
        sample_ques, # ques
        sample_ans, # given answer
        "", # output - leave this blank for generation! LLM willl generate is it is True or False
    )

print("Input Promt:\n", input_prompt)
inputs = tokenizer(
[
    input_prompt
], return_tensors = "pt").to("cuda")

input_shape = inputs['input_ids'].shape
input_token_len = input_shape[1] # 1 because of batch
outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
# you can get the whole generated text by uncommenting the below line
# text_generated = tokenizer.batch_decode([outputs, skip_special_tokens=True)

response = tokenizer.batch_decode([outputs[0][input_token_len:]], skip_special_tokens=True)
response

Input Promt:
 You are a great mathematician and you are tasked with finding if an answer to a given maths question is correct or not. Yout response should be 'True' if correct, otherwise 'False'. Below is Question and Answer.



### Question:
The Parker family needs to leave the house by 5 pm for a dinner party. Mrs. Parker was waiting to get into the bathroom at 2:30 pm. Her oldest daughter used the bathroom for 45 minutes and her youngest daughter used the bathroom for another 30 minutes. Then her husband used it for 20 minutes. How much time will Mrs. Parker have to use the bathroom to leave on time?

### Answer:
205

### Explainaition

### Output:



['True']

## saving model

In [ ]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

In [ ]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference


==((====))==  Unsloth 2024.11.5: Fast Llama patching. Transformers = 4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu124. CUDA = 7.5. CUDA Toolkit = 12.4.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
